In [450]:
import pandas as pd
import pyspark as spark
from pyspark.sql import *
from pyspark.sql import SparkSession
from datetime import datetime
from datetime import timedelta

In [451]:
xls = pd.ExcelFile('combined.xlsx')
accident_df = pd.read_excel(xls, 'Accident Data')
accident_df = accident_df.astype({"Location": "string", "timestamp": "string",\
                  "accident": 'string'})

traffic_df = pd.read_excel(xls, 'Traffic Data')
traffic_df = traffic_df.astype({"Location": "string", "Date": "string",\
                  "start_time": 'string', 'Traffic': 'string'})

In [452]:
# display(traffic_df, accident_df)

In [453]:
# traffic_df['timestamp'] = traffic_df['Date'] + ' ' + traffic_df['start_time']

# traffic_df

# traffic_df.pop('Date')
# traffic_df.pop('start_time')
# traffic_df

In [454]:
# #Building Spark Session
# spark = SparkSession.builder \
#     .master("local[1]") \
#     .appName("testApp") \
#     .getOrCreate()



In [455]:
# from pyspark.sql.types import *
# from pyspark.sql import SQLContext
# from pyspark import SparkContext

# sc = SparkContext.getOrCreate()
# sqlContext = SQLContext(sc)


In [456]:

# # Auxiliar functions
# def equivalent_type(f):
#     if f == 'datetime64[ns]': return TimestampType()
#     elif f == 'int64': return LongType()
#     elif f == 'int32': return IntegerType()
#     elif f == 'float64': return FloatType()
#     else: return StringType()

# def define_structure(string, format_type):
#     try: typo = equivalent_type(format_type)
#     except: typo = StringType()
#     return StructField(string, typo)

# # Given pandas dataframe, it will return a spark's dataframe.
# def pandas_to_spark(pandas_df):
#     columns = list(pandas_df.columns)
#     types = list(pandas_df.dtypes)
#     struct_list = []
#     for column, typo in zip(columns, types): 
#       struct_list.append(define_structure(column, typo))
#     p_schema = StructType(struct_list)
#     return sqlContext.createDataFrame(pandas_df, p_schema)

In [457]:
# # Trying out operations

# Location = traffic_df[(traffic_df.Location == 'H1') & (traffic_df.Date == '2021-01-01') & (traffic_df.start_time == '02:30:00')].Location
# Location = Location.to_string()[2:]
# Location = Location.replace(' ', '')

# timeInterval = (traffic_df[(traffic_df.Location == 'H1') & (traffic_df.Date == '2021-01-01') & (traffic_df.start_time == '02:30:00')].Date) + ' ' +\
#      (traffic_df[(traffic_df.Location == 'H1') & (traffic_df.Date == '2021-01-01') & (traffic_df.start_time == '02:30:00')].start_time)

# date = timeInterval.to_string()[2:]
# date = date.replace(' ', '')[0:10]
# # print(date)
# time = timeInterval.to_string()[2:]
# time = time.replace(' ', '')[10:15]
# # print(time)
# day = datetime.datetime.strptime(date, '%Y-%m-%d').weekday()


# # print(type(timeInterval))

# if day == 5 or day == 6:
#      is_weekday = 0
# else:
#      is_weekday = 1

# # 9-11 AM and 5-6 PM on weekdaysonly.

# if is_weekday == 1:
#      t = datetime.datetime.strptime(time, '%H:%M')
#      # print(t)
#      morning_t1 = datetime.datetime.strptime('09:00', '%H:%M')
#      morning_t2 = datetime.datetime.strptime('11:00', '%H:%M')
#      evening_t1 = datetime.datetime.strptime('17:00', '%H:%M')
#      evening_t2 = datetime.datetime.strptime('18:00', '%H:%M')
#      if (t >= morning_t1 and t <= morning_t2) or (t >= evening_t1 and t <= evening_t2):
#           is_rushhour = 1
#      else:
#           is_rushhour = 0


# # l1 = Location[Location.start_time == '00:30:00']
# # print(Location.dtypes)
# # print(Location)
# # print(timeInterval.to_string()[5:15])
# # print(type(day))
# # print(is_rushhour)
# # print(timeInterval.to_string()[17:21])
# # day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
# # print(day_name[is_weekday]) 

# print('Location: ', Location)
# print('Time Interval: ', date + ' ' + time)
# print('Is Weekday: ', is_weekday)
# print('Is Rush hour: ', is_rushhour)



In [458]:
#concatinating the Date and Start_time column to timestamp and removing them
traffic_df['timestamp'] = traffic_df['Date'] + ' ' + traffic_df['start_time']

traffic_df.pop('Date')
traffic_df.pop('start_time')
# traffic_df

0       00:00:00
1       00:30:00
2       01:00:00
3       01:30:00
4       02:00:00
          ...   
2147    21:30:00
2148    22:00:00
2149    22:30:00
2150    23:00:00
2151    23:30:00
Name: start_time, Length: 2152, dtype: string

In [459]:
#Changing the datatype of Traffic column to float for further operations.
traffic_df = traffic_df.astype({'Traffic': 'float'})

#Inputting the difference om the new Traffic_Dif column and filling NaN value with 0 -- To be handled in next stage.
traffic_df['traffic_diff'] = traffic_df['Traffic'].diff().fillna(0)

In [460]:
# #Cleaning & Convertion to single float data
# flowDiff = traffic_df[(traffic_df.Location == 'H1') & (traffic_df.timestamp == ('2021-01-01' + ' ' + '02:30:00'))].traffic_dif
# flowDiff = flowDiff.to_string()[2:]
# flowDiff = flowDiff.replace(' ', '')
# # flowDiff

In [461]:
#Creating a DF that contains the max of last 120 mins (4 windows)
max_last_120 = traffic_df['Traffic'].rolling(window=4).max().shift(0).fillna(0)
# max_last_120

In [462]:
#Stacking the max_120 DF nest to Traffic DF and renaming columns
horizontal_stack = pd.concat([traffic_df, max_last_120], axis=1)
horizontal_stack.columns = ['Location','Traffic', 'timestamp', 'traffic_diff', 'maxLast_120']
traffic_df = horizontal_stack
traffic_df

,Location,Traffic,timestamp,traffic_diff,maxLast_120
0,H1,51.0,2021-01-01 00:00:00,0.0,0.0
1,H1,3.0,2021-01-01 00:30:00,-48.0,0.0
2,H1,99.0,2021-01-01 01:00:00,96.0,0.0
3,H1,57.0,2021-01-01 01:30:00,-42.0,99.0
4,H1,66.0,2021-01-01 02:00:00,9.0,99.0
...,...,...,...,...,...
2147,H5,72.0,2021-01-09 21:30:00,58.0,72.0
2148,H5,15.0,2021-01-09 22:00:00,-57.0,72.0
2149,H5,95.0,2021-01-09 22:30:00,80.0,95.0
2150,H5,93.0,2021-01-09 23:00:00,-2.0,95.0


In [463]:
for i,row in traffic_df.iterrows():
    datetime1 = datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')
    datetime2 = datetime1 + timedelta(minutes=30)
    # accident_count = accident_df[(accident_df.timestamp >= str(datetime1)) & (accident_df.timestamp <= str(datetime2))].accident.count()
    traffic_df.loc[i, 'timestamp'] = str(datetime2)

In [464]:
#Creating new columns and setting the values to None
traffic_df['is_rushhour'] = None
traffic_df['is_weekday'] = None

In [465]:
#Creating the Final DF
for i, row in traffic_df.iterrows():
     
     day = datetime.strptime(row['timestamp'][:10], '%Y-%m-%d').weekday()
     if day == 5 or day == 6:
          is_weekday = 0
          traffic_df.loc[i,'is_weekday'] = 0          
     else:
          is_weekday = 1
          traffic_df.loc[i,'is_weekday'] = 1

     if is_weekday == 1:
     # print(row['timestamp'][11:16])
          t = datetime.strptime(row['timestamp'][11:16], '%H:%M')
          # print(t)
          morning_t1 = datetime.strptime('09:00', '%H:%M')
          morning_t2 = datetime.strptime('11:00', '%H:%M')
          evening_t1 = datetime.strptime('17:00', '%H:%M')
          evening_t2 = datetime.strptime('18:00', '%H:%M')
          if (t >= morning_t1 and t <= morning_t2) or (t >= evening_t1 and t <= evening_t2):
               is_rushhour = 1
               traffic_df.loc[i, 'is_rushhour'] = 1
          else:
               is_rushhour = 0
               traffic_df.loc[i, 'is_rushhour'] = 0
     else:
          traffic_df.loc[i, 'is_rushhour'] = 0

In [466]:
traffic_df
traffic_df_final = traffic_df[['Location', 'timestamp', 'is_weekday', 'is_rushhour', 'traffic_diff', 'maxLast_120']]

In [467]:
traffic_df_final['no_of_accidents'] = None

In [468]:
accident_df['timestamp'] = accident_df['timestamp'].map(lambda x: str(x)[:-4])
# accident_df

In [469]:
for i,row in traffic_df_final.iterrows():
    datetime1 = datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')
    datetime2 = datetime1 + timedelta(minutes=15)
    accident_count = accident_df[(accident_df.timestamp >= str(datetime1)) & (accident_df.timestamp <= str(datetime2))].accident.count()
    traffic_df_final.loc[i, 'no_of_accidents'] = accident_count

In [470]:
traffic_df_final.columns = ['Location', 'Time Interval', 'Is Weekday?', 'Is Rushhour?', 'Flow Diff Same Location', 'Max Last 120 Same Location', 'Accidents_Next_15']

,Location,Time Interval,Is Weekday?,Is Rushhour?,Flow Diff Same Location,Max Last 120 Same Location,Accidents_Next_15
0,H1,2021-01-01 00:30:00,1,0,0.0,0.0,0
1,H1,2021-01-01 01:00:00,1,0,-48.0,0.0,0
2,H1,2021-01-01 01:30:00,1,0,96.0,0.0,0
3,H1,2021-01-01 02:00:00,1,0,-42.0,99.0,0
4,H1,2021-01-01 02:30:00,1,0,9.0,99.0,1
5,H1,2021-01-01 03:00:00,1,0,35.0,101.0,0
6,H1,2021-01-01 03:30:00,1,0,-11.0,101.0,0
7,H1,2021-01-01 04:00:00,1,0,-82.0,101.0,0
8,H1,2021-01-01 04:30:00,1,0,90.0,101.0,0
9,H1,2021-01-01 05:00:00,1,0,-69.0,98.0,0
